<a href="https://colab.research.google.com/github/jtsou/Amazon-Recommendation-Engine/blob/main/ISYE_6740.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import seaborn
from matplotlib import pyplot as plt
import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Electronics.csv

--2022-04-12 01:04:14--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Electronics.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 876247561 (836M) [application/octet-stream]
Saving to: ‘Electronics.csv’

Electronics.csv     100%[===================>] 835.65M  38.8MB/s    in 22s     

2022-04-12 01:04:36 (38.0 MB/s) - ‘Electronics.csv’ saved [876247561/876247561]



In [ ]:
electronics_rating = pd.read_csv('Electronics.csv', header= None)

In [ ]:
electronics_rating.columns = ['item','user','rating','timestamp']
electronics_rating.head()

,item,user,rating,timestamp
0,0060009810,A1N070NS9CJQ2I,5.0,1026864000
1,0060009810,A3P0KRKOBQK1KN,5.0,1025913600
2,0060009810,A192HO2ICJ75VU,5.0,1025654400
3,0060009810,A2T278FKFL3BLT,4.0,1025395200
4,0060009810,A2ZUXVTW8RXBXW,5.0,1025222400


In [ ]:
electronics_rating['timestamp'] = pd.to_datetime(electronics_rating['timestamp'],unit='s')


In [ ]:
electronics_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20994353 entries, 0 to 20994352
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   item       object        
 1   user       object        
 2   rating     float64       
 3   timestamp  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 640.7+ MB


In [ ]:
electronics_rating.timestamp.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                20994353
unique                   7015
top       2016-03-01 00:00:00
freq                    25244
first     1997-12-04 00:00:00
last      2018-10-05 00:00:00
Name: timestamp, dtype: object

In [ ]:
electronics_rating.item.describe()

count       20994353
unique        756489
top       B010OYASRG
freq           28539
Name: item, dtype: object

In [ ]:
electronics_rating.user.describe()

count          20994353
unique          9838676
top       A680RUE1FDO8B
freq                633
Name: user, dtype: object

In [ ]:
electronics_rating.groupby('user')['rating'].mean()

user
A0000040I1OM9N4SGBD8    2.000000
A00010809P09NUU6ZP6H    5.000000
A0001392JPQ8N7MSF2CV    5.000000
A0001410YRCTKL1MHXQJ    4.000000
A0001528BGUBOEVR6T5U    4.428571
                          ...   
AZZZY1W55XHZR           4.000000
AZZZYCYV82JER           2.000000
AZZZZ4BWKERJ9           4.000000
AZZZZHRHTLH4R           1.000000
AZZZZJYGA32             3.000000
Name: rating, Length: 9838676, dtype: float64

In [ ]:
## This is for KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
## This is for SVD